# DB 데이터 추출

In [12]:
from plugins import db_connector
import pandas as pd

conn, cursor = db_connector.mysql()
cursor.execute('select * from restaurant;')
df = pd.DataFrame(cursor.fetchall())
df = df[[0,1,2,3]]
df[3] = df[3].str.split(' ').str[:4].str.join(' ')

# 위경도 데이터 추출

In [ ]:
import json
import requests

api_key = 'aec8af9f13176cd0b2a600a395e57953'
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

for idx in df.index:
    try:
        id = df.loc[idx, 0]
        mapx, mapy = addr_to_lat_lon(df.loc[idx,3])
        df.loc[idx, 'mapx'] = mapx
        df.loc[idx, 'mapy'] = mapy
    except:
        df.loc[idx, 'mapx'] = None
        df.loc[idx, 'mapy'] = None
        pass
    
df.to_csv('./data/레스토랑위치정보.csv', index = False, encoding = 'utf-8-sig')

# DB 업데이트

In [29]:
cursor.execute('BEGIN;')
try:
    for idx in df.index:
        id = df.loc[idx, 0]
        mapx = df.loc[idx, 'mapx'] 
        mapy = df.loc[idx, 'mapy']
        sql = f'UPDATE restaurant SET mapx = {mapx}, mapy = {mapy} WHERE id = {id};'
        cursor.execute(sql)
    cursor.execute('COMMIT;')
except Exception as e:
    print(e)
    cursor.execute('ROLLBACK;')
    raise

# NULL 값 삭제

In [35]:
# df.to_csv('./data/레스토랑위치정보.csv', index = False, encoding = 'utf-8-sig')
df = pd.read_csv('C:/Stand_Up_Seoul/data/retaurant/data/레스토랑위치정보.csv')
df = df[df['mapx'].isnull()]

cursor.execute('BEGIN;')
try:
    for idx in df.index:
        id = df.loc[idx, '0']
        # mapx = df.loc[idx, 'mapx'] 
        # mapy = df.loc[idx, 'mapy']
        sql = f'DELETE FROM restaurant WHERE id = {id};'
        cursor.execute(sql)
    cursor.execute('COMMIT;')
except Exception as e:
    print(e)
    cursor.execute('ROLLBACK;')
    raise
